# Save and restore models

Model progress can be saved during — and after — training. It is good practice to share:
* code to create a model
* the trained weights and parameters

How to save a model depends on the API used

In [1]:
from __future__ import absolute_import, division, print_function

import os

import tensorflow as tf
from tensorflow import keras

tf.__version__

'1.13.1'

#### Loading data

In [2]:
(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.mnist.load_data()

train_labels = train_labels[:1000]
test_labels = test_labels[:1000]

train_images = train_images[:1000].reshape(-1, 28 * 28) / 255.0 # -1 infers the shape => 1000 in this case (nb of examples)
test_images = test_images[:1000].reshape(-1, 28 * 28) / 255.0

#### Building a simple model

This model uses sparse categorical crosentropy, because the target labels are integers. If we used categorical crossentropy we would need to encode them with one-hot.

In [3]:
# Returns a short sequential model
def create_model():
  model = tf.keras.models.Sequential([
    keras.layers.Dense(512, activation=tf.keras.activations.relu, input_shape=(784,)),
    keras.layers.Dropout(0.2),
    keras.layers.Dense(10, activation=tf.keras.activations.softmax)
  ])
  
  model.compile(optimizer=tf.keras.optimizers.Adam(),
                loss=tf.keras.losses.sparse_categorical_crossentropy,
                metrics=['accuracy'])
  
  return model


# Create a basic model instance
model = create_model()
model.summary()

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 512)               401920    
_________________________________________________________________
dropout (Dropout)            (None, 512)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 10)                5130      
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


## Save checkpoints during training

The primary use case is to automatically save checkpoints during and at the end of training. This way we can use a trained model without having to retrain it, or pick-up training where you left of—in case the training process was interrupted.

In [4]:
checkpoint_path = "training_1/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

# Create checkpoint callback
cp_callback = tf.keras.callbacks.ModelCheckpoint(checkpoint_path, 
                                                 save_weights_only=True,
                                                 verbose=1)

model = create_model()

model.fit(train_images, train_labels,  epochs = 10, 
          validation_data = (test_images,test_labels),
          callbacks = [cp_callback])  # pass callback to training

# This may generate warnings related to saving the state of the optimizer.
# These warnings (and similar warnings throughout this notebook)
# are in place to discourage outdated usage, and can be ignored.

Train on 1000 samples, validate on 1000 samples
Epoch 1/10
 832/1000 [=======================>......] - ETA: 0s - loss: 1.2387 - acc: 0.6550
Epoch 00001: saving model to training_1/cp.ckpt

Consider using a TensorFlow optimizer from `tf.train`.
Instructions for updating:
Use tf.train.CheckpointManager to manage checkpoints rather than manually editing the Checkpoint proto.
1000/1000 [==============================] - 1s 729us/sample - loss: 1.1304 - acc: 0.6840 - val_loss: 0.7394 - val_acc: 0.7710
Epoch 2/10
 832/1000 [=======================>......] - ETA: 0s - loss: 0.4234 - acc: 0.8774
Epoch 00002: saving model to training_1/cp.ckpt

Consider using a TensorFlow optimizer from `tf.train`.
1000/1000 [==============================] - 0s 486us/sample - loss: 0.4248 - acc: 0.8810 - val_loss: 0.5245 - val_acc: 0.8270
Epoch 3/10
 832/1000 [=======================>......] - ETA: 0s - loss: 0.2829 - acc: 0.9267
Epoch 00003: saving model to training_1/cp.ckpt

Consider using a TensorFlow opt

In [5]:
model = create_model()

loss, acc = model.evaluate(test_images, test_labels)
print("Untrained model, accuracy: {:5.2f}%".format(100*acc))

1000/1000 [==============================] - 0s 101us/sample - loss: 2.3425 - acc: 0.0810
Untrained model, accuracy:  8.10%


In [6]:
model.load_weights(checkpoint_path)

loss,acc = model.evaluate(test_images, test_labels)
print("Restored model, accuracy: {:5.2f}%".format(100*acc))

1000/1000 [==============================] - 0s 44us/sample - loss: 0.4086 - acc: 0.8640
Restored model, accuracy: 86.40%


In [7]:
# include the epoch in the file name. (uses `str.format`)
checkpoint_path = "training_2/cp-{epoch:04d}.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

cp_callback = tf.keras.callbacks.ModelCheckpoint(
    checkpoint_path, verbose=1, save_weights_only=True,
    # Save weights, every 5-epochs.
    period=5)

model = create_model()
model.save_weights(checkpoint_path.format(epoch=0))
model.fit(train_images, train_labels,
          epochs = 50, callbacks = [cp_callback],
          validation_data = (test_images,test_labels),
          verbose=0)


Consider using a TensorFlow optimizer from `tf.train`.

Epoch 00005: saving model to training_2/cp-0005.ckpt

Consider using a TensorFlow optimizer from `tf.train`.

Epoch 00010: saving model to training_2/cp-0010.ckpt

Consider using a TensorFlow optimizer from `tf.train`.

Epoch 00015: saving model to training_2/cp-0015.ckpt

Consider using a TensorFlow optimizer from `tf.train`.

Epoch 00020: saving model to training_2/cp-0020.ckpt

Consider using a TensorFlow optimizer from `tf.train`.

Epoch 00025: saving model to training_2/cp-0025.ckpt

Consider using a TensorFlow optimizer from `tf.train`.

Epoch 00030: saving model to training_2/cp-0030.ckpt

Consider using a TensorFlow optimizer from `tf.train`.

Epoch 00035: saving model to training_2/cp-0035.ckpt

Consider using a TensorFlow optimizer from `tf.train`.

Epoch 00040: saving model to training_2/cp-0040.ckpt

Consider using a TensorFlow optimizer from `tf.train`.

Epoch 00045: saving model to training_2/cp-0045.ckpt

Consider 

In [8]:
! dir {checkpoint_dir}

 Volume in drive D is DATA
 Volume Serial Number is 881A-0976

 Directory of D:\Projetos\Learning\TensorFlowPy\training_2

19/05/2019  14:27    <DIR>          .
19/05/2019  14:27    <DIR>          ..
19/05/2019  14:27                81 checkpoint
19/05/2019  14:27         1ÿ631ÿ508 cp-0000.ckpt.data-00000-of-00001
19/05/2019  14:27               648 cp-0000.ckpt.index
19/05/2019  14:27         1ÿ631ÿ508 cp-0005.ckpt.data-00000-of-00001
19/05/2019  14:27               648 cp-0005.ckpt.index
19/05/2019  14:27         1ÿ631ÿ508 cp-0010.ckpt.data-00000-of-00001
19/05/2019  14:27               648 cp-0010.ckpt.index
19/05/2019  14:27         1ÿ631ÿ508 cp-0015.ckpt.data-00000-of-00001
19/05/2019  14:27               648 cp-0015.ckpt.index
19/05/2019  14:27         1ÿ631ÿ508 cp-0020.ckpt.data-00000-of-00001
19/05/2019  14:27               648 cp-0020.ckpt.index
19/05/2019  14:27         1ÿ631ÿ508 cp-0025.ckpt.data-00000-of-00001
19/05/2019  14:27               648 cp-0025.ckpt.index
19/05/201

Note: the default tensorflow format only saves the 5 most recent checkpoints.

In [9]:
latest = tf.train.latest_checkpoint(checkpoint_dir)
latest

'training_2\\cp-0050.ckpt'

In [10]:
model = create_model()
model.load_weights(latest)
loss, acc = model.evaluate(test_images, test_labels)
print("Restored model, accuracy: {:5.2f}%".format(100*acc))

1000/1000 [==============================] - 0s 104us/sample - loss: 0.4854 - acc: 0.8760
Restored model, accuracy: 87.60%


## Manually save weigths

In [11]:
# Save the weights
model.save_weights('./checkpoints/my_checkpoint')

# Restore the weights
model = create_model()
model.load_weights('./checkpoints/my_checkpoint')

loss,acc = model.evaluate(test_images, test_labels)
print("Restored model, accuracy: {:5.2f}%".format(100*acc))


Consider using a TensorFlow optimizer from `tf.train`.
1000/1000 [==============================] - 0s 98us/sample - loss: 0.4854 - acc: 0.8760
Restored model, accuracy: 87.60%


##  Save the entire model

The entire model can be saved to a file that contains:
* the weight values
* the model's configuration
* the optimizer's configuration (depends on set up). 

This allows you to checkpoint a model and resume training later—from the exact same state—without access to the original code.

Saving a fully-functional model is very useful—you can load them in TensorFlow.js (HDF5, Saved Model) and then train and run them in web browsers, or convert them to run on mobile devices using TensorFlow Lite (HDF5, Saved Model)

####  HDF5 file

In [12]:
model = create_model()

model.fit(train_images, train_labels, epochs=5)

# Save entire model to a HDF5 file
model.save('my_model.h5')

Epoch 1/5
1000/1000 [==============================] - 0s 431us/sample - loss: 1.1932 - acc: 0.6510
Epoch 2/5
1000/1000 [==============================] - 0s 292us/sample - loss: 0.4321 - acc: 0.8740
Epoch 3/5
1000/1000 [==============================] - 0s 296us/sample - loss: 0.2748 - acc: 0.9290
Epoch 4/5
1000/1000 [==============================] - 0s 325us/sample - loss: 0.2056 - acc: 0.9560
Epoch 5/5
1000/1000 [==============================] - 0s 296us/sample - loss: 0.1509 - acc: 0.9680


In [13]:
# Recreate the exact same model, including weights and optimizer.
new_model = keras.models.load_model('my_model.h5')
new_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_12 (Dense)             (None, 512)               401920    
_________________________________________________________________
dropout_6 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_13 (Dense)             (None, 10)                5130      
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


In [14]:
loss, acc = new_model.evaluate(test_images, test_labels)
print("Restored model, accuracy: {:5.2f}%".format(100*acc))

1000/1000 [==============================] - 0s 117us/sample - loss: 0.4445 - acc: 0.8530
Restored model, accuracy: 85.30%


Currently, it is not able to save TensorFlow optimizers (from tf.train).

#### saved_model

In [15]:
model = create_model()

model.fit(train_images, train_labels, epochs=5)

Epoch 1/5
1000/1000 [==============================] - 0s 397us/sample - loss: 1.0744 - acc: 0.6890
Epoch 2/5
1000/1000 [==============================] - 0s 311us/sample - loss: 0.4041 - acc: 0.8930
Epoch 3/5
1000/1000 [==============================] - 0s 396us/sample - loss: 0.2795 - acc: 0.9310
Epoch 4/5
1000/1000 [==============================] - 0s 490us/sample - loss: 0.2083 - acc: 0.9440
Epoch 5/5
1000/1000 [==============================] - 0s 314us/sample - loss: 0.1539 - acc: 0.9660


In [20]:
saved_model_path = tf.contrib.saved_model.save_keras_model(model, "./saved_models")


Consider using a TensorFlow optimizer from `tf.train`.
Instructions for updating:
This function will only be available through the v1 compatibility library as tf.compat.v1.saved_model.utils.build_tensor_info or tf.compat.v1.saved_model.build_tensor_info.
INFO:tensorflow:Signatures INCLUDED in export for Classify: None
INFO:tensorflow:Signatures INCLUDED in export for Regress: None
INFO:tensorflow:Signatures INCLUDED in export for Predict: ['serving_default']
INFO:tensorflow:Signatures INCLUDED in export for Train: None
INFO:tensorflow:Signatures INCLUDED in export for Eval: None
INFO:tensorflow:No assets to save.
INFO:tensorflow:No assets to write.
INFO:tensorflow:SavedModel written to: ./saved_models\1558271627\saved_model.pb


In [21]:
new_model = tf.contrib.saved_model.load_keras_model(saved_model_path)
new_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_14 (Dense)             (None, 512)               401920    
_________________________________________________________________
dropout_7 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_15 (Dense)             (None, 10)                5130      
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


In [22]:
# The model has to be compiled before evaluating.
# This step is not required if the saved model is only being deployed.

new_model.compile(optimizer=tf.keras.optimizers.Adam(), 
              loss=tf.keras.losses.sparse_categorical_crossentropy,
              metrics=['accuracy'])

# Evaluate the restored model.
loss, acc = new_model.evaluate(test_images, test_labels)
print("Restored model, accuracy: {:5.2f}%".format(100*acc))

1000/1000 [==============================] - 0s 126us/sample - loss: 0.4345 - acc: 0.8670
Restored model, accuracy: 86.70%
